In [4]:
import json
# from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset
import numpy as np
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [5]:
with open("data/editorial_main.json", "r") as read_file:
    data = json.load(read_file)

df = pd.DataFrame(data)

In [6]:
train_dataset = Dataset.from_pandas(df)

In [7]:
# mean length of body: 1345.997
# mean length of third-option: 30.59

In [19]:
model_name = 'tuner007/pegasus_paraphrase'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to('cuda')

In [22]:
def get_response(input_text,num_return_sequences):
    batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to('cuda')
    translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [35]:
neg_thesis = []
for i in data: neg_thesis.append(i['neg-thesis'])

In [36]:
summarized_neg_options = []
for i in tqdm(neg_thesis):
    summarized_neg_options.append(get_response(i, 1))

100%|██████████| 841/841 [04:43<00:00,  2.97it/s]


In [37]:
for idx, k in enumerate(data):
    k['neg-thesis'] = summarized_neg_options[idx]

In [38]:
for i in range(len(data)):
    assert data[i]['neg-thesis'] == summarized_neg_options[i]

In [39]:
with open('/home/nlp/apex/transformer-editorial/synthetic_neg.json', 'w') as fout:
    json.dump(data , fout)